# Time horizon analysis of the GWP, EGWP, GTP and ATR

_Dataset_: Supplementary data for Megill et al. (2024): "Alternative climate metrics to the Global Warming Potential are more suitable for assessing aviation non-CO2 effects"

_Authors_:

- Liam Megill (1, 2), https://orcid.org/0000-0002-4199-6962   
- Kathrin Deck (2)  
- Volker Grewe (1, 2), https://orcid.org/0000-0002-8012-6783  

_Affiliation (1)_: Deutsches Zentrum für Luft- und Raumfahrt (DLR), Institut für Physik der Atmosphäre, Oberpfaffenhofen, Germany

_Affiliation (2)_: Delft University of Technology (TU Delft), Faculty of Aerospace Engineering, Section Aircraft Noise and Climate Effects (ANCE), Delft, The Netherlands

_Corresponding author_: Liam Megill, liam.megill@dlr.de

_doi_: https://doi.org/10.1038/s43247-024-01423-6

---


### Summary
This notebook analyses the Global Warming Potential (GWP), "Effective" Global Warming Potential (EGWP), Global Temperature Change Potential (GTP) and Average Temperature Response (ATR) with respect to the time horizon (Figure 4 and Extended Data Figures 3, 4, 5, 6 in linked paper).

### Linked data
- `GEN/F2020`: Standard fleet emission with starting year 2020
- `GEN/P2020`: Pulse emission with starting year 2020
- `GEN/INC1`: 1% increasing emission with starting year 2020
- `GEN/E_bg_fuel_GEN.txt`: Emission scenario input file for AirClim

The other emission scenarios in `GEN` were used in research towards the linked article, but are not included in this analysis.

---

### Copyright

Copyright © 2024 Liam Megill

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0. Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

# define plotting style
colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']
sns.set_context("paper", rc={"lines.linewidth": 2})
sns.set_style("ticks")

## Function definition

In [ ]:
def plot_metric_response(metric, abs_or_rel,
                         fig, ax_pls, ax_flt, ax_inc, 
                         all_species=True, ssp_lines=True, linestyle="-"):
    """Plot the metric response vs time horizon.

    Args:
        metric (_str_): Metric name: "GWP", "EGWP", "ATR", "GTP"
        abs_or_rel (_str_): Kind of metric: "abs", "relCO2", "reltot"
        fig (_matplotlib.figure.Figure_): Figure
        ax_pls (_matplotlib.axes._axes.Axes_): Axis for pulse response
        ax_flt (_matplotlib.axes._axes.Axes_): Axis for fleet response
        ax_inc (_matplotlib.axes._axes.Axes_): Axis for increasing response
        all_species (_bool_): Plot all species (True) or just total (False)
        ssp_lines (_bool_): Show outlines of SSP range (True) or not (False)
        linestyle (_str_): Linestyle to be used for plotting
    """
    
    # initialise arrays
    max_res = np.zeros((7, 101))
    min_res = np.zeros((7, 101))
    res_SSP2 = np.zeros((7, 101))

    for i_scen, (scenario, ax) in enumerate(zip(scenarios, (ax_pls, ax_flt, ax_inc))):
        for bg_name in bg_names:
            # Load RF and dT
            folder = "../data/GEN/%s/%s/" % (scenario, bg_name)
            res = np.zeros((7, 101))
            rf_res = np.zeros((7, 101))
            rf_full = np.zeros((7, len(years)))
            dt_res = np.zeros((7, 101))
            for i in range(1, 7):  # total RF (ges, idx 0) not available in data, so is calculated
                rf_file = "RF_%s_taumean_rfmean.txt" % emission_species[i]
                rf_full[i, :] = np.loadtxt(folder + rf_file, skiprows=2, usecols=1)
                rf_res[i, :] = rf_full[i, yr_idxs]
            rf_full[0, :] = np.sum(rf_full[1:, :], axis=0)
            rf_res[0, :] = np.sum(rf_res[1:, :], axis=0)
            for i in range(7):
                dt_file = "dt_%s_taumean_rfmean_lammean.txt" % emission_species[i]
                dt_res[i, :] = np.loadtxt(folder + dt_file, skiprows=2, usecols=1)[yr_idxs]
            
            # select metric calculation method
            if metric == "GWP":
                res = np.array([[np.sum(rf_res[j, 0:H]) for H in range(101)] for j in range(7)]) / 1000.  # mW -> W
            elif metric == "EGWP":
                efficacies = np.array([1., 1.14, 1.37, 1.18, 0.59, 1.])  # efficacies from Ponater et al. (2006)
                res = np.array([[np.sum(rf_res[j, 0:H]) for H in range(101)] for j in range(7)]) / 1000.
                res[1:, :] = res[1:, :] * efficacies.reshape((6, 1))
                res[0, :] = np.sum(res[1:, :], axis=0)
            elif metric == "GTP": 
                res = np.array([[dt_res[j, int(H)] for H in range(101)] for j in range(7)])
            elif metric == "ATR":
                res = np.array([[np.sum(dt_res[j, 0:H]) / H for H in range(101)] for j in range(7)])
            else:
                print("metric name unknown")
            
            # select absolute or relative
            if abs_or_rel == "abs":
                res = res
            elif abs_or_rel == "relCO2":
                res = res / res[1, :]
            elif abs_or_rel == "reltot":
                res = res / res[0, :]

            # initialise different SSPs
            if bg_name == bg_names[0]:
                max_res = res
                min_res = res
            else:
                max_res = np.maximum(max_res, res)
                min_res = np.minimum(min_res, res)
            if bg_name == "SSP2-45":
                res_SSP2 = res
        
        # Plot each species i
        i_lst = range(7) if all_species else [0]
        for i in i_lst:
            ax.fill_between(range(1, 101), min_res[i, 1:], max_res[i, 1:], facecolor=colors[i], alpha=0.1)
            if ssp_lines:  # plot ssp outlines
                ax.plot(range(1, 101), min_res[i, 1:], color=colors[i], linestyle='--', linewidth=0.8, alpha=0.8)
                ax.plot(range(1, 101), max_res[i, 1:], color=colors[i], linestyle='--', linewidth=0.8, alpha=0.8)
            ax.plot(range(1, 101), res_SSP2[i, 1:], color=colors[i], label=emission_labels[i], linestyle=linestyle)

        for item in ([ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
            item.set_fontsize(14)


## Plot comparison of GWP, EGWP and ATR responses

This section creates Figure 4 of the linked paper. Uncomment the last line to save the figure to the `images` folder.

In [ ]:
# initialise figure
fig, ((ax_plsr, ax_fltr, ax_incr), (ax_plsrt, ax_fltrt, ax_incrt)) = plt.subplots(nrows=2, ncols=3,
                                                                                  sharex="all", sharey="row",
                                                                                  figsize=(13, 7))

# setup
emission_species = ["Ges", "CO2", "H2O", "O3", "CH4", "Cont", "PMO"]
emission_labels = ["Total", r"CO$_2$", r"H$_2$O", r"O$_3$", r"CH$_4$", "Contrails", "PMO"]
bg_names = ["SSP1-19", "SSP2-45", "SSP3-7", "SSP4-6", "SSP5-85"]
metrics = ["GWP", "EGWP", "rATR & iGTP"]
linestyles = ['-', '--', ':']
scenarios = ["P2020", "F2020", "INC1"]
years = np.loadtxt("../data/GEN/P2020/SSP1-19/dt_CO2_taumean_rfmean_lammean.txt", skiprows=2, usecols=0, dtype=int)
start_year = 2020
yr_idxs = np.array([i for i, e in enumerate(years) if start_year <= e <= start_year + 100])

# plot metric responses
plot_metric_response("GWP", "relCO2",
                     fig, ax_plsr, ax_fltr, ax_incr, False, False, linestyles[0])
plot_metric_response("GWP", "reltot",
                     fig, ax_plsrt, ax_fltrt, ax_incrt, True, False, linestyles[0])
plot_metric_response("EGWP", "relCO2",
                     fig, ax_plsr, ax_fltr, ax_incr, False, False, linestyles[1])
plot_metric_response("EGWP", "reltot",
                     fig, ax_plsrt, ax_fltrt, ax_incrt, True, False, linestyles[1])
plot_metric_response("ATR", "relCO2",
                     fig, ax_plsr, ax_fltr, ax_incr, False, False, linestyles[2])
plot_metric_response("ATR", "reltot",
                     fig, ax_plsrt, ax_fltrt, ax_incrt, True, False, linestyles[2])

# set axis sizes and labels
ax_plsr.set_xlim([0, 100])
ax_plsr.set_ylim([0, 14])
ax_plsrt.set_ylim([-0.3, 1.1])
ax_fltrt.set_xlabel("Time Horizon [yr]")
ax_plsr.set_ylabel("Relative to CO$_2$ [-]")
ax_plsrt.set_ylabel("Relative to total [-]")

# add metric legend
met_legend_elements = [
    Line2D([0], [0], linestyle=linestyles[0], linewidth=2, label=metrics[0]),
    Line2D([0], [0], linestyle=linestyles[1], linewidth=2, label=metrics[1]),
    Line2D([0], [0], linestyle=linestyles[2], linewidth=2, label=metrics[2]),
]
met_legend = ax_incr.legend(handles=met_legend_elements, ncol=1, loc="lower left", handlelength=3, fontsize=11)

# add species legend
species_legend_elements = [
    Line2D([0], [0], color=color, linewidth=2, label=label) for color, label in zip(colors, emission_labels)
]
species_legend = ax_fltr.legend(handles=species_legend_elements, ncol=1, loc="lower left", fontsize=11)


# plot emission profiles in inset axes
fuel_yrs = np.loadtxt("../data/GEN/E_bg_fuel_GEN.txt", skiprows=3, usecols=0)
fuel_data = np.loadtxt("../data/GEN/E_bg_fuel_GEN.txt", skiprows=3, usecols=[1, 8, 5])
insax_pls = ax_plsr.inset_axes([0.5, 0.7, 0.45, 0.25])
insax_flt = ax_fltr.inset_axes([0.5, 0.7, 0.45, 0.25], sharex=insax_pls, sharey=insax_pls)
insax_inc = ax_incr.inset_axes([0.5, 0.7, 0.45, 0.25], sharex=insax_pls, sharey=insax_pls)
insax_pls.plot(fuel_yrs, fuel_data[:, 0] * 716.16272, "k")
insax_flt.plot(fuel_yrs, fuel_data[:, 1] * 716.16272, "k")
insax_inc.plot(fuel_yrs, fuel_data[:, 2] * 716.16272, "k")
insax_pls.set_xlim([2000, 2120])
insax_pls.set_ylim([0, 2000])
insax_pls.set_xlabel("Year")
insax_pls.set_ylabel("Fuel use [Tg]")
for ax in (insax_flt, insax_inc):
    ax.xaxis.set_tick_params(which="both", labelbottom=False, labeltop=False)
    ax.yaxis.set_tick_params(which="both", labelleft=False, labelright=False)

# add subfigure labels
fig.text(0.11, 0.92, "a", fontsize=24)
fig.text(0.39, 0.92, "b", fontsize=24)
fig.text(0.70, 0.92, "c", fontsize=24)
fig.text(0.08, 0.44, "d", fontsize=24)
fig.text(0.39, 0.44, "e", fontsize=24)
fig.text(0.70, 0.44, "f", fontsize=24)

# configure and save figure
fig.tight_layout()
fig_savename = "../images/analysis_gwp-egwp-atr.png"
# fig.savefig(fig_savename, dpi=600)

## Plot GWP response

This section creates Extended Data Figure 3 of the linked paper. Uncomment the last line to save the figure to the `images` folder.

In [ ]:
# initialise figure
fig = plt.figure(figsize=(13, 10))
ax_pls = fig.add_subplot(331)
ax_flt = fig.add_subplot(332, sharex=ax_pls)
ax_inc = fig.add_subplot(333, sharex=ax_pls)
ax_plsr = fig.add_subplot(334, sharex=ax_pls)
ax_fltr = fig.add_subplot(335, sharex=ax_pls, sharey=ax_plsr)
ax_incr = fig.add_subplot(336, sharex=ax_pls, sharey=ax_plsr)
ax_plsrt = fig.add_subplot(337, sharex=ax_pls)
ax_fltrt = fig.add_subplot(338, sharex=ax_pls, sharey=ax_plsrt)
ax_incrt = fig.add_subplot(339, sharex=ax_pls, sharey=ax_plsrt)

# setup
emission_species = ["Ges", "CO2", "H2O", "O3", "CH4", "Cont", "PMO"]
emission_labels = ["Total", r"CO$_2$", r"H$_2$O", r"O$_3$", r"CH$_4$", "Contrails", "PMO"]
bg_names = ["SSP1-19", "SSP2-45", "SSP3-7", "SSP4-6", "SSP5-85"]
scenarios = ["P2020", "F2020", "INC1"]
years = np.loadtxt("../data/GEN/P2020/SSP1-19/dt_CO2_taumean_rfmean_lammean.txt", skiprows=2, usecols=0, dtype=int)
start_year = 2020
yr_idxs = np.array([i for i, e in enumerate(years) if start_year <= e <= start_year + 100])

# plot responses
plot_metric_response("GWP", "abs", fig, ax_pls, ax_flt, ax_inc)
plot_metric_response("GWP", "relCO2", fig, ax_plsr, ax_fltr, ax_incr)
plot_metric_response("GWP", "reltot", fig, ax_plsrt, ax_fltrt, ax_incrt)

# set axis labels and limits
ax_pls.set_ylabel("AGWP [W m$^{-2}$ yr]", fontsize=16)
ax_plsr.set_ylabel("GWP [-]", fontsize=16)
ax_plsrt.set_ylabel("AGWP / AGWP$_{tot}$ [-]", fontsize=16)
ax_inc.legend(loc="upper left", ncol=2, fontsize=10)
for ax in (ax_pls, ax_flt, ax_inc, ax_plsr, ax_fltr, ax_incr):
    ax.xaxis.set_tick_params(which="both", labelbottom=False, labeltop=False)
    ax.set_xlabel("")
ax_pls.set_xlim([0, 100])
ax_pls.set_ylim([-0.1, 0.5]); ax_flt.set_ylim([-5, 20]); ax_inc.set_ylim([-10, 50])
ax_plsr.set_ylim([-2, 14]); ax_fltr.set_ylim([-2, 14]); ax_incr.set_ylim([-2, 14])
ax_plsrt.set_ylim([-0.3, 1.1]); ax_fltrt.set_ylim([-0.3, 1.1]); ax_incrt.set_ylim([-0.3, 1.1])
ax_fltrt.set_xlabel("Time Horizon [yr]", fontsize=16)

# plot emission profiles in inset axes
fuel_yrs = np.loadtxt("../data/GEN/E_bg_fuel_GEN.txt", skiprows=3, usecols=0)
fuel_data = np.loadtxt("../data/GEN/E_bg_fuel_GEN.txt", skiprows=3, usecols=[1, 8, 5])
insax_pls = ax_plsr.inset_axes([0.5, 0.7, 0.45, 0.25])
insax_flt = ax_fltr.inset_axes([0.5, 0.7, 0.45, 0.25], sharex=insax_pls, sharey=insax_pls)
insax_inc = ax_incr.inset_axes([0.5, 0.7, 0.45, 0.25], sharex=insax_pls, sharey=insax_pls)
insax_pls.plot(fuel_yrs, fuel_data[:, 0] * 716.16272, "k")
insax_flt.plot(fuel_yrs, fuel_data[:, 1] * 716.16272, "k")
insax_inc.plot(fuel_yrs, fuel_data[:, 2] * 716.16272, "k")
insax_pls.set_xlim([2000, 2120])
insax_pls.set_ylim([0, 2000])
insax_pls.set_xlabel("Year")
insax_pls.set_ylabel("Fuel use [Tg]")
for ax in (insax_flt, insax_inc):
    ax.xaxis.set_tick_params(which="both", labelbottom=False, labeltop=False)
    ax.yaxis.set_tick_params(which="both", labelleft=False, labelright=False)

# add subfigure labels
fig.text(0.025, 0.95, "a", fontsize=24)
fig.text(0.345, 0.95, "b", fontsize=24)
fig.text(0.670, 0.95, "c", fontsize=24)
fig.text(0.025, 0.63, "d", fontsize=24)
fig.text(0.345, 0.63, "e", fontsize=24)
fig.text(0.670, 0.63, "f", fontsize=24)
fig.text(0.025, 0.31, "g", fontsize=24)
fig.text(0.345, 0.31, "h", fontsize=24)
fig.text(0.670, 0.31, "i", fontsize=24)

# configure and save figure
fig.tight_layout()
fig_savename = "../images/gwp_analysis.png"
# fig.savefig(fig_savename, dpi=600)

## Plot EGWP response

This section creates Extended Data Figure 4 of the linked paper. Uncomment the last line to save the figure to the `images` folder.

In [ ]:
# initialise figure
fig = plt.figure(figsize=(13, 10))
ax_pls = fig.add_subplot(331)
ax_flt = fig.add_subplot(332, sharex=ax_pls)
ax_inc = fig.add_subplot(333, sharex=ax_pls)
ax_plsr = fig.add_subplot(334, sharex=ax_pls)
ax_fltr = fig.add_subplot(335, sharex=ax_pls, sharey=ax_plsr)
ax_incr = fig.add_subplot(336, sharex=ax_pls, sharey=ax_plsr)
ax_plsrt = fig.add_subplot(337, sharex=ax_pls)
ax_fltrt = fig.add_subplot(338, sharex=ax_pls, sharey=ax_plsrt)
ax_incrt = fig.add_subplot(339, sharex=ax_pls, sharey=ax_plsrt)

# setup
emission_species = ["Ges", "CO2", "H2O", "O3", "CH4", "Cont", "PMO"]
emission_labels = ["Total", r"CO$_2$", r"H$_2$O", r"O$_3$", r"CH$_4$", "Contrails", "PMO"]
bg_names = ["SSP1-19", "SSP2-45", "SSP3-7", "SSP4-6", "SSP5-85"]
scenarios = ["P2020", "F2020", "INC1"]
years = np.loadtxt("../data/GEN/P2020/SSP1-19/dt_CO2_taumean_rfmean_lammean.txt", skiprows=2, usecols=0, dtype=int)
start_year = 2020
yr_idxs = np.array([i for i, e in enumerate(years) if start_year <= e <= start_year + 100])

# plot responses
plot_metric_response("EGWP", "abs", fig, ax_pls, ax_flt, ax_inc)
plot_metric_response("EGWP", "relCO2", fig, ax_plsr, ax_fltr, ax_incr)
plot_metric_response("EGWP", "reltot", fig, ax_plsrt, ax_fltrt, ax_incrt)

# set axis labels and limits
ax_pls.set_ylabel("AEGWP [W m$^{-2}$ yr]", fontsize=16)
ax_plsr.set_ylabel("EGWP [-]", fontsize=16)
ax_plsrt.set_ylabel("AEGWP / AEGWP$_{tot}$ [-]", fontsize=16)
ax_inc.legend(loc="upper left", ncol=2, fontsize=10)
for ax in (ax_pls, ax_flt, ax_inc, ax_plsr, ax_fltr, ax_incr):
    ax.xaxis.set_tick_params(which="both", labelbottom=False, labeltop=False)
    ax.set_xlabel("")
ax_pls.set_xlim([0, 100])
ax_pls.set_ylim([-0.1, 0.5]); ax_flt.set_ylim([-5, 20]); ax_inc.set_ylim([-10, 50])
ax_plsr.set_ylim([-2, 14]); ax_fltr.set_ylim([-2, 14]); ax_incr.set_ylim([-2, 14])
ax_plsrt.set_ylim([-0.3, 1.1]); ax_fltrt.set_ylim([-0.3, 1.1]); ax_incrt.set_ylim([-0.3, 1.1])
ax_fltrt.set_xlabel("Time Horizon [yr]", fontsize=16)

# plot emission profiles in inset axes
fuel_yrs = np.loadtxt("../data/GEN/E_bg_fuel_GEN.txt", skiprows=3, usecols=0)
fuel_data = np.loadtxt("../data/GEN/E_bg_fuel_GEN.txt", skiprows=3, usecols=[1, 8, 5])
insax_pls = ax_plsr.inset_axes([0.5, 0.7, 0.45, 0.25])
insax_flt = ax_fltr.inset_axes([0.5, 0.7, 0.45, 0.25], sharex=insax_pls, sharey=insax_pls)
insax_inc = ax_incr.inset_axes([0.5, 0.7, 0.45, 0.25], sharex=insax_pls, sharey=insax_pls)
insax_pls.plot(fuel_yrs, fuel_data[:, 0] * 716.16272, "k")
insax_flt.plot(fuel_yrs, fuel_data[:, 1] * 716.16272, "k")
insax_inc.plot(fuel_yrs, fuel_data[:, 2] * 716.16272, "k")
insax_pls.set_xlim([2000, 2120])
insax_pls.set_ylim([0, 2000])
insax_pls.set_xlabel("Year")
insax_pls.set_ylabel("Fuel use [Tg]")
for ax in (insax_flt, insax_inc):
    ax.xaxis.set_tick_params(which="both", labelbottom=False, labeltop=False)
    ax.yaxis.set_tick_params(which="both", labelleft=False, labelright=False)

# add subfigure labels
fig.text(0.025, 0.95, "a", fontsize=24)
fig.text(0.345, 0.95, "b", fontsize=24)
fig.text(0.670, 0.95, "c", fontsize=24)
fig.text(0.025, 0.63, "d", fontsize=24)
fig.text(0.345, 0.63, "e", fontsize=24)
fig.text(0.670, 0.63, "f", fontsize=24)
fig.text(0.025, 0.31, "g", fontsize=24)
fig.text(0.345, 0.31, "h", fontsize=24)
fig.text(0.670, 0.31, "i", fontsize=24)

# configure and save figure
fig.tight_layout()
fig_savename = "../images/egwp_analysis.png"
# fig.savefig(fig_savename, dpi=600)

## Plot ATR response

This section creates Extended Data Figure 5 of the linked paper. Uncomment the last line to save the figure to the `images` folder.

In [ ]:
# initialise figure
fig = plt.figure(figsize=(13, 10))
ax_pls = fig.add_subplot(331)
ax_flt = fig.add_subplot(332, sharex=ax_pls)
ax_inc = fig.add_subplot(333, sharex=ax_pls)
ax_plsr = fig.add_subplot(334, sharex=ax_pls)
ax_fltr = fig.add_subplot(335, sharex=ax_pls, sharey=ax_plsr)
ax_incr = fig.add_subplot(336, sharex=ax_pls, sharey=ax_plsr)
ax_plsrt = fig.add_subplot(337, sharex=ax_pls)
ax_fltrt = fig.add_subplot(338, sharex=ax_pls, sharey=ax_plsrt)
ax_incrt = fig.add_subplot(339, sharex=ax_pls, sharey=ax_plsrt)

# setup
emission_species = ["Ges", "CO2", "H2O", "O3", "CH4", "Cont", "PMO"]
emission_labels = ["Total", r"CO$_2$", r"H$_2$O", r"O$_3$", r"CH$_4$", "Contrails", "PMO"]
bg_names = ["SSP1-19", "SSP2-45", "SSP3-7", "SSP4-6", "SSP5-85"]
scenarios = ["P2020", "F2020", "INC1"]
years = np.loadtxt("../data/GEN/P2020/SSP1-19/dt_CO2_taumean_rfmean_lammean.txt", skiprows=2, usecols=0, dtype=int)
start_year = 2020
yr_idxs = np.array([i for i, e in enumerate(years) if start_year <= e <= start_year + 100])

# plot responses
plot_metric_response("ATR", "abs", fig, ax_pls, ax_flt, ax_inc)
plot_metric_response("ATR", "relCO2", fig, ax_plsr, ax_fltr, ax_incr)
plot_metric_response("ATR", "reltot", fig, ax_plsrt, ax_fltrt, ax_incrt)

# set axis labels and limits
ax_pls.set_ylabel("ATR [mK]", fontsize=16)
ax_plsr.set_ylabel("rATR [-]", fontsize=16)
ax_plsrt.set_ylabel("ATR / ATR$_{tot}$ [-]", fontsize=16)
ax_inc.legend(loc="upper left", ncol=2, fontsize=10)
for ax in (ax_pls, ax_flt, ax_inc, ax_plsr, ax_fltr, ax_incr):
    ax.xaxis.set_tick_params(which="both", labelbottom=False, labeltop=False)
    ax.set_xlabel("")
ax_pls.set_xlim([0, 100])
ax_pls.set_ylim([-1, 5]); ax_flt.set_ylim([-20, 80]); ax_inc.set_ylim([-50, 200])
ax_plsr.set_ylim([-2, 14]); ax_fltr.set_ylim([-2, 14]); ax_incr.set_ylim([-2, 14])
ax_plsrt.set_ylim([-0.3, 1.1]); ax_fltrt.set_ylim([-0.3, 1.1]); ax_incrt.set_ylim([-0.3, 1.1])
ax_fltrt.set_xlabel("Time Horizon [yr]", fontsize=16)

# plot emission profiles in inset axes
fuel_yrs = np.loadtxt("../data/GEN/E_bg_fuel_GEN.txt", skiprows=3, usecols=0)
fuel_data = np.loadtxt("../data/GEN/E_bg_fuel_GEN.txt", skiprows=3, usecols=[1, 8, 5])
insax_pls = ax_plsr.inset_axes([0.5, 0.7, 0.45, 0.25])
insax_flt = ax_fltr.inset_axes([0.5, 0.7, 0.45, 0.25], sharex=insax_pls, sharey=insax_pls)
insax_inc = ax_incr.inset_axes([0.5, 0.7, 0.45, 0.25], sharex=insax_pls, sharey=insax_pls)
insax_pls.plot(fuel_yrs, fuel_data[:, 0] * 716.16272, "k")
insax_flt.plot(fuel_yrs, fuel_data[:, 1] * 716.16272, "k")
insax_inc.plot(fuel_yrs, fuel_data[:, 2] * 716.16272, "k")
insax_pls.set_xlim([2000, 2120])
insax_pls.set_ylim([0, 2000])
insax_pls.set_xlabel("Year")
insax_pls.set_ylabel("Fuel use [Tg]")
for ax in (insax_flt, insax_inc):
    ax.xaxis.set_tick_params(which="both", labelbottom=False, labeltop=False)
    ax.yaxis.set_tick_params(which="both", labelleft=False, labelright=False)

# add subfigure labels
fig.text(0.025, 0.95, "a", fontsize=24)
fig.text(0.345, 0.95, "b", fontsize=24)
fig.text(0.670, 0.95, "c", fontsize=24)
fig.text(0.025, 0.63, "d", fontsize=24)
fig.text(0.345, 0.63, "e", fontsize=24)
fig.text(0.670, 0.63, "f", fontsize=24)
fig.text(0.025, 0.31, "g", fontsize=24)
fig.text(0.345, 0.31, "h", fontsize=24)
fig.text(0.670, 0.31, "i", fontsize=24)

# configure and save figure
fig.tight_layout()
fig_savename = "../images/atr_analysis.png"
# fig.savefig(fig_savename, dpi=600)

## Plot GTP response

This section creates Extended Data Figure 6 of the linked paper. Uncomment the last line to save the figure to the `images` folder.

In [ ]:
# initialise figure
fig = plt.figure(figsize=(13, 10))
ax_pls = fig.add_subplot(331)
ax_flt = fig.add_subplot(332, sharex=ax_pls)
ax_inc = fig.add_subplot(333, sharex=ax_pls)
ax_plsr = fig.add_subplot(334, sharex=ax_pls)
ax_fltr = fig.add_subplot(335, sharex=ax_pls, sharey=ax_plsr)
ax_incr = fig.add_subplot(336, sharex=ax_pls, sharey=ax_plsr)
ax_plsrt = fig.add_subplot(337, sharex=ax_pls)
ax_fltrt = fig.add_subplot(338, sharex=ax_pls, sharey=ax_plsrt)
ax_incrt = fig.add_subplot(339, sharex=ax_pls, sharey=ax_plsrt)

# setup
emission_species = ["Ges", "CO2", "H2O", "O3", "CH4", "Cont", "PMO"]
emission_labels = ["Total", r"CO$_2$", r"H$_2$O", r"O$_3$", r"CH$_4$", "Contrails", "PMO"]
bg_names = ["SSP1-19", "SSP2-45", "SSP3-7", "SSP4-6", "SSP5-85"]
scenarios = ["P2020", "F2020", "INC1"]
years = np.loadtxt("../data/GEN/P2020/SSP1-19/dt_CO2_taumean_rfmean_lammean.txt", skiprows=2, usecols=0, dtype=int)
start_year = 2020
yr_idxs = np.array([i for i, e in enumerate(years) if start_year <= e <= start_year + 100])

# plot responses
plot_metric_response("GTP", "abs", fig, ax_pls, ax_flt, ax_inc)
plot_metric_response("GTP", "relCO2", fig, ax_plsr, ax_fltr, ax_incr)
plot_metric_response("GTP", "reltot", fig, ax_plsrt, ax_fltrt, ax_incrt)

# set axis labels and limits
ax_pls.set_ylabel("AGTP [mK]", fontsize=16)
ax_plsr.set_ylabel("GTP [-]", fontsize=16)
ax_plsrt.set_ylabel("AGTP / AGTP$_{tot}$ [-]", fontsize=16)
ax_inc.legend(loc="upper left", ncol=2, fontsize=10)
for ax in (ax_pls, ax_flt, ax_inc, ax_plsr, ax_fltr, ax_incr):
    ax.xaxis.set_tick_params(which="both", labelbottom=False, labeltop=False)
    ax.set_xlabel("")
ax_pls.set_xlim([0, 100])
ax_pls.set_ylim([-1, 5]); ax_flt.set_ylim([-40, 120]); ax_inc.set_ylim([-100, 400])
ax_plsr.set_ylim([-2, 14]); ax_fltr.set_ylim([-2, 14]); ax_incr.set_ylim([-2, 14])
ax_plsrt.set_ylim([-0.3, 1.1]); ax_fltrt.set_ylim([-0.3, 1.1]); ax_incrt.set_ylim([-0.3, 1.1])
ax_fltrt.set_xlabel("Time Horizon [yr]", fontsize=16)

# plot emission profiles in inset axes
fuel_yrs = np.loadtxt("../data/GEN/E_bg_fuel_GEN.txt", skiprows=3, usecols=0)
fuel_data = np.loadtxt("../data/GEN/E_bg_fuel_GEN.txt", skiprows=3, usecols=[1, 8, 5])
insax_pls = ax_plsr.inset_axes([0.5, 0.7, 0.45, 0.25])
insax_flt = ax_fltr.inset_axes([0.5, 0.7, 0.45, 0.25], sharex=insax_pls, sharey=insax_pls)
insax_inc = ax_incr.inset_axes([0.5, 0.7, 0.45, 0.25], sharex=insax_pls, sharey=insax_pls)
insax_pls.plot(fuel_yrs, fuel_data[:, 0] * 716.16272, "k")
insax_flt.plot(fuel_yrs, fuel_data[:, 1] * 716.16272, "k")
insax_inc.plot(fuel_yrs, fuel_data[:, 2] * 716.16272, "k")
insax_pls.set_xlim([2000, 2120])
insax_pls.set_ylim([0, 2000])
insax_pls.set_xlabel("Year")
insax_pls.set_ylabel("Fuel use [Tg]")
for ax in (insax_flt, insax_inc):
    ax.xaxis.set_tick_params(which="both", labelbottom=False, labeltop=False)
    ax.yaxis.set_tick_params(which="both", labelleft=False, labelright=False)

# add subfigure labels
fig.text(0.025, 0.95, "a", fontsize=24)
fig.text(0.345, 0.95, "b", fontsize=24)
fig.text(0.670, 0.95, "c", fontsize=24)
fig.text(0.025, 0.63, "d", fontsize=24)
fig.text(0.345, 0.63, "e", fontsize=24)
fig.text(0.670, 0.63, "f", fontsize=24)
fig.text(0.025, 0.31, "g", fontsize=24)
fig.text(0.345, 0.31, "h", fontsize=24)
fig.text(0.670, 0.31, "i", fontsize=24)

# configure and save figure
fig.tight_layout()
fig_savename = "../images/gtp_analysis.png"
# fig.savefig(fig_savename, dpi=600)